# Hometask 03
### Maksim Sorokin

#### Реализовать с помощью объектно-ориентированного подхода предыдущие 2 задания. Создайте для каждой из задач отдельный класс, который позволяет ее решить.

Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
Cохранить ее в pandas dataframe, а также в excel, pickle, БД

Проверить состояние датафрейма и привести его в формат, позволяющий дальнейшее исследование данных

Сделать визуализацию для мини-исследования рынка строящейся недвижимости в одном или нескольких регионах с помощью pandas, matplotlib, seaborn, plotly и других инструментов.

In [13]:
import requests
import pandas as pd
import openpyxl
from tqdm import tqdm
from pandas import DataFrame
from sqlalchemy import create_engine
import os.path

In [14]:
class DomRfRestClient:

    def getTotal(self):
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset=1&limit=100&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
        res = requests.get(url)
        return res.json().get('data').get('total')

    def getIds(self,offset_ = 1, limit_ = 100):
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
        res = requests.get(url)
        objects_data = res.json()
        objects_data.get('data').get('list')[0]
        objects_list = objects_data.get('data').get('list')
        l = list()
        for a in objects_list:
            l.append(a['objId'])
        return l

    def get_object_by_id(self, id):
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{id}'
        answ = requests.get(url)
        return answ.json()
    

In [15]:
class DomRfDataLoader:

    client: DomRfRestClient
    
    def __init__(self,a:DomRfRestClient):
        self.client = a

    def get_all_ids(self):
        all_objects_list = list()
        work_in_progress = True
        offset = 1
        total_objects =  self.client.getTotal()
        iterations = total_objects // 100
        print('Expected iterations count: ')
        if (total_objects % 100) > 0:
            iterations+=1

        for a in tqdm(range(iterations)):
            all_objects_list.extend(self.client.getIds(a*100))

        assert len(all_objects_list) == total_objects, 'Didn\'t load the whole dataset'
        print('Loaded '+str(len(all_objects_list)) + ' entries, total '+str(total_objects))
        return all_objects_list

    def get_all_objects(self):
        all_objects_full_list = list()
        ids_with_errors = list()
        for a in tqdm(self.get_all_ids()):
            try:
                all_objects_full_list.append(self.client.get_object_by_id(a))
            except:
                ids_with_errors.append(a)

        print('Additional processing of failed ids')
        for b in tqdm(ids_with_errors):
            all_objects_full_list.append(self.client.get_object_by_id(b))

        assert len(all_objects_full_list) == self.client.getTotal(), 'Didn\'t load the whole dataset'
        return all_objects_full_list

In [22]:
class DomRfDataSaver:
    elements: list
    internal_df: DataFrame
    
    def __init__(self, list):
        self.elements = list
        self.internal_df = DataFrame()
        for c in tqdm(self.elements):
            temp_result_dict = dict(c.get('data'))
            developer_dict = temp_result_dict.get('developer')
            if developer_dict:
                devId = developer_dict.get('devId')
                del temp_result_dict['developer']
                temp_result_dict['devId'] = devId
            temp_df = DataFrame([temp_result_dict])
        self.internal_df = pd.concat([self.internal_df,temp_df], ignore_index = True)

    def save_csv(self, file_name: str):
        self.internal_df.to_csv(file_name)

    def save_xlsx(self, file_name: str):
        self.internal_df.to_excel(file_name)

    def save_sql(self, file_name: str):
        result_df_str =  self.internal_df.applymap(str)
        engine = create_engine('sqlite://', echo=False)
        result_df_str.to_sql(file_name, con=engine)
            

In [17]:
if not os.path.exists('output2/dom_objects.xlsx'):
    restClient = DomRfRestClient()
    loader = DomRfDataLoader(restClient)
    all_houses_list = loader.get_all_objects()

Expected iterations count: 


100%|██████████| 10/10 [00:13<00:00,  1.38s/it]


Loaded 1000 entries, total 1000


100%|██████████| 1000/1000 [02:15<00:00,  7.40it/s]


Additional processing of failed ids


0it [00:00, ?it/s]


In [23]:
dataSaver = DomRfDataSaver(all_houses_list)
dataSaver.save_csv('output2/dom_objects.csv')
dataSaver.save_xlsx('output2/dom_objects.xlsx')
dataSaver.save_sql('output_2/dom_objects')


100%|██████████| 1000/1000 [00:00<00:00, 1357.91it/s]
